In [1]:
import numpy as np

In [2]:
# data I/O
data = open('shakespear.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [16]:
data_size, vocab_size

(99993, 62)

In [4]:
test1 = {}
test2 = [1,2,3]

In [6]:
test1[-1] = np.copy(test2)

In [10]:
test1[-1]

array([1, 2, 3])

In [4]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  print(np.shape(xs[t]), np.shape(hs[t]), np.shape(ps[t]))
  
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [5]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n', txt, '\n----')

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter', n, 'loss: ', smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 BPJ?I
qC';-ASKYda?'VWymp
cqjo!!KOvWU's?Jwwk!mxRFx:SoGhjfjkp-:NelnCXwupmufVisMaziLghPqxZPXJWGTe;SsHswKErYV-fB;RCe:C!.eUlIWtnbaFGZ,YCNsTShU:mjCY
PrW.A'o
bbjuqXTzEgL- K
',XIvYbXjjRr'us:k
vWf:CtbCEcWLFoQm 
----
(62, 1) (100, 1) (62, 1)
iter 0 loss:  103.17836498588026
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)
(62, 1) (100, 1) (62, 1)


In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n', txt, '\n----')

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter', n, 'loss: ', smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 GvlJ?ybuIEWi,Qwf RiOQ,hltWsxhwTIJzARyUNyUEmw.MlszR-DFxP:GIE-FjYBy!AZ-tMLqJapTLLM.'Qoi,Ip?xVN gy:ofdBo?VrvAS;RTYAqW;XZW'.BANazygHJsl 'enzuBcaJ''OLTwlRKmF;WAj'JKgOupfIj?fAmL.!vk,w'rsGZogGgYlWwyBnqBAsrmz 
----
iter 0 loss:  103.17835964644293
----
 hhc: T e
et oh.nd oOh edehnd snlOhr SMI
cuatB'ltmof Ks'teu:vfl a  ,snirTmhoeosoh

r onUr Sm
uIfednchdoehnid tn h
a.aoahisvehh:o.taRryinUdHt o Tag nafjIwkuigi.:d etm
BtDt
Bptht,he a teaIioe  eoSgI a  h 
----
iter 100 loss:  103.55383808429643
----
 Rneoa,hranhe edti uioli hyaeeeg e:,oxhea tststdDhBoo idwnit  lu
tur se hdwroInT!oeI nCou lhtoskaloooot,
dme crj mww
GrmeLboeh'rnu;i idoarednoosey sl  isetNhtmoe ngosIeaeaom; ealc ndRot wd
b:htphswfl i 
----
iter 200 loss:  102.09374919207738
----
  mhhsilfynug :he rf en ytmosiotns S fhthyam oshaeet  a eer lei ip  ,
r  ty rmee rs
melretduomddwEgthIiwl pe ithoiir.pu eemo s treleene aeskwr
ns,  fhmt.l oenruehtr  tetraliruaahruh wwYaryaDn b agtorrs 
----
iter 300 loss:  100.44805164561184
----
 ts n